Предполагается, что в рамках данного задания ученики сами будут изучать документацию, как минимум используя функционал `tab` и `shift+tab` от Jupyter

1. Импортируйте библиотеку pandas 
2. Считайте файл `wage.csv` в DataFrame с названием `wage`. Используйте для этого функцию pd.read_csv()

`person_id` - идентификатор человека

`gender` - пол, 0 - женский, 1 - мужской

`wage` - заработная плата в рублях

In [ ]:
import pandas as pd

In [1]:
wage = pd.read_csv('https://edu.tinkoff.ru/files/ee13a6b1-605e-44fe-83a6-85d3d7a0b3ae')

In [7]:
wage.head()

,person_id,gender,wage
0,0,1,46793.603811
1,1,1,33481.575720
2,2,1,44523.699084
3,3,1,15995.576829
4,4,0,10282.631224


3. Поменяйте колонку `gender` так, чтобы были записи `F` (female) и `M` (male) вместо 0 и 1

In [10]:
wage['gender'] = wage['gender'].apply(lambda x:'F' if x == 0 else 'M')

4. Давайте посчитаем среднюю зарплату у мужчин и женщин. Для этого используйте метод `groupby` со следующим синтаксисом:

`dataframe.groupby(Название_колонки_для_группировки)[Перечисление_колонок_для_агрегации].функция_агрегации()`

In [11]:
wage.groupby('gender')['wage'].sum()

gender
F    2.034616e+07
M    2.378250e+07
Name: wage, dtype: float64

5. Теперь взглянем внимательнее на данные и обнаружим, что некоторые люди попали в выборку несколько раз. 
    1. Найдите таких людей. Подсказка: `value_counts()`
    0. Убедитесь, что записи по ним с одинаковым `wage`. Возможно, тут вам пригодится функция агрегации `nunique()`, отображающая количество разных значений
    0. Избавьтесь от повторяющихся значений. Подсказка: `drop_duplicates()`

In [31]:
wage['person_id'].value_counts()

17     2
16     2
15     2
14     2
13     2
      ..
342    1
343    1
344    1
345    1
999    1
Name: person_id, Length: 1000, dtype: int64

In [27]:
wage['wage'].nunique()

1000

In [32]:
wage.drop_duplicates(subset=['wage','person_id'], inplace= True)

6. Теперь посмотрим внимательнее на зарплаты
    1. Охарактеризуйте имеющиеся данные по зарплатам. Подсказка: `describe`
    1. Избавьтесь от бессмысленных значений

In [34]:
wage['wage'].describe()

count      1000.000000
mean      43694.227404
std       55352.539343
min     -287418.645743
25%       14489.682367
50%       27309.529498
75%       52021.080258
max      755320.874132
Name: wage, dtype: float64

In [36]:
wage = wage[wage['wage'] >= 0]

7. Давайте теперь посмотрим на зарплату с учетом бонуса. Для этого нам понадобится таблица `bonus.csv`. Считайте ее в переменную `bonus`. Заметьте, что она сохранена немного в другом формате, и вам понадобится уточнить параметр `sep` - разделитель записей. Сравните текущий файл с предыдущим и попробуйте решить проблему

In [39]:
bonus = pd.read_csv('https://edu.tinkoff.ru/files/c177edc7-6e8e-4e53-8f5d-098bd95cc4de', sep = ';')

In [40]:
bonus.head()

,person_id,bonus
0,905,85059.638382
1,836,7703.346074
2,287,3120.269742
3,548,5347.987142
4,575,137257.490614


8. Чтобы посчитать итоговую зарплату, нам нужно по каждому человеку знать и оклад, и премию. Для этого надо будет соединить (сджойнить) таблицы по `person_id`. Используйте для этого функцию `pd.merge`. Помните, что параметр `how` должен быть `'outer'`, чтобы сохранить те записи, что есть только в одной таблице. Результат запишите в новый dataframe `df`

In [41]:
df = wage.merge(bonus, how = 'outer', on = 'person_id')

In [43]:
df.head()

,person_id,gender,wage,bonus
0,0,M,46793.603811,3.332934e+04
1,1,M,33481.575720,NaN
2,2,M,44523.699084,3.192912e+06
3,3,M,15995.576829,2.196858e+04
4,4,F,10282.631224,NaN


9. Наконец, давайте посчитаем итоговую зарплату
    1. Замените отсутствующие записи в колонке `bonus` нулями
    1. Уберите людей без `wage` - это те "плохие" записи, от которых мы избавлялись на предыдущих шагах
    1. Сделайте новую колонку `total`, которая будет равна 12 окладам и премии
    1. Посчитайте среднюю и медианную итоговую зарплату в разрезе по полу. Подсказка: вместо функции агрегации можно написать `.agg()` и перечислить внутри нужные агрегаты

In [45]:
df['bonus'] = df['bonus'].fillna(0)

In [48]:
df['wage'] = df['wage'].dropna()

In [49]:
df['total'] = 12 * df['wage'] + df['bonus']

In [53]:
df.groupby('gender')['total'].agg(['median','mean']).reset_index()

,gender,median,mean
0,F,347622.913892,570746.139432
1,M,437499.824868,657142.490282


10. Сохраните `df` в файл, используя метод `to_csv()`. Не записывайте индексы

In [54]:
df.to_csv('tinkoff_case.csv')